In [13]:
import pandas as pd
import re
import string
import torch
import transformers
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from sklearn.manifold import TSNE
from nltk.tokenize import word_tokenize
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


<h2><strong> Partie II : FakeNewsChallenge (Deep Learning + Machine Learning) <strong></h2> 

<h3><strong> Data Exploration </strong></h3>

| rows  | unrelated | discuss  | agree    | disagree |
|-------|-----------|----------|----------|----------|
| 49972 | 0.73131   | 0.17828  | 0.0736012| 0.0168094|


In [1]:
import pandas as pd
label = pd.read_csv("fnc-1/train_stances.csv")

In [2]:
label.head()

,Headline,Body ID,Stance
0,Police find mass graves with at least '15 bodi...,712,unrelated
1,Hundreds of Palestinians flee floods in Gaza a...,158,agree
2,"Christian Bale passes on role of Steve Jobs, a...",137,unrelated
3,HBO and Apple in Talks for $15/Month Apple TV ...,1034,unrelated
4,Spider burrowed through tourist's stomach and ...,1923,disagree


In [3]:
label.shape

(49972, 3)

In [4]:
array = label['Body ID'].unique()
print(len(array))

1683


In [5]:
body = pd.read_csv("fnc-1/train_bodies.csv")

In [6]:
body.shape

(1683, 2)

In [7]:
label.columns

Index(['Headline', 'Body ID', 'Stance'], dtype='object')

In [24]:
import pandas as pd

# Supprimer la colonne 'Body ID'
label = label.drop('Body ID', axis=1)

# Renommer 'Headline' en 'text' et 'Stance' en 'labels'
label = label.rename(columns={'Headline': 'text', 'Stance': 'labels'})

In [25]:
from sklearn.preprocessing import LabelEncoder

# Créer une instance de LabelEncoder
label_encoder = LabelEncoder()

# Appliquer le label encoding sur la colonne 'labels'
label['labels'] = label_encoder.fit_transform(label['labels'])


In [26]:
data = label

In [27]:
from datasets import Dataset
dataset = Dataset.from_pandas(data)

<h3><strong> Regression Logistique </strong></h3>

In [28]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


# Séparer les données en ensemble d'entraînement et de test
train_df, test_df = train_test_split(data, test_size=0.2, random_state=42)

# Créer un vecteurisateur Bag of Words
vectorizer = CountVectorizer(max_features=5000)  # Nombre maximum de mots à prendre en compte

# Transformer les textes en vecteurs
X_train = vectorizer.fit_transform(train_df['text'])
X_test = vectorizer.transform(test_df['text'])

# Définir les étiquettes
y_train = train_df['labels']
y_test = test_df['labels']

# Créer un modèle de régression logistique
model = LogisticRegression()

# Entraîner le modèle
model.fit(X_train, y_train)

# Faire des prédictions
predictions = model.predict(X_test)

# Calculer l'accuracy
accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy}")


Accuracy: 0.7275637818909455


C:\Users\CYTech Student\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


La précision n'est pas représentative de l'efficacité du modèle.

<h2><strong> Deep Learning (LSTM) </strong></h2>

In [29]:
import pandas as pd
label = pd.read_csv("fnc-1/train_stances.csv")
body = pd.read_csv("fnc-1/train_bodies.csv")

In [30]:
data = pd.merge(label, body, on='Body ID', how='inner')

# how='inner' signifie que seules les lignes ayant des valeurs correspondantes dans les deux dataframes seront incluses.
# Si vous voulez garder toutes les lignes de df1 et df2, utilisez how='outer'.

# Afficher le résultat
data.head()

,Headline,Body ID,Stance,articleBody
0,Police find mass graves with at least '15 bodi...,712,unrelated,Danny Boyle is directing the untitled film\n\n...
1,Seth Rogen to Play Apple’s Steve Wozniak,712,discuss,Danny Boyle is directing the untitled film\n\n...
2,Mexico police find mass grave near site 43 stu...,712,unrelated,Danny Boyle is directing the untitled film\n\n...
3,Mexico Says Missing Students Not Found In Firs...,712,unrelated,Danny Boyle is directing the untitled film\n\n...
4,New iOS 8 bug can delete all of your iCloud do...,712,unrelated,Danny Boyle is directing the untitled film\n\n...


In [31]:
data.shape

(49972, 4)

In [32]:
from sklearn.preprocessing import LabelEncoder

# Créer une instance de LabelEncoder
label_encoder = LabelEncoder()

# Appliquer le label encoding sur la colonne 'labels'
data['Stance'] = label_encoder.fit_transform(data['Stance'])

In [33]:
import pandas as pd
from sklearn.utils import shuffle

# Charger votre dataset (remplacez "votre_dataset.csv" par le chemin de votre fichier)
dataset = data

# Compter le nombre d'échantillons par classe
counts = dataset['Stance'].value_counts()

# Trouver la classe minoritaire
minority_class = counts.idxmin()

# Déterminer le nombre d'échantillons dans la classe minoritaire
minority_count = counts.min()

# Sous-échantillonner les classes majoritaires pour égaler la classe minoritaire
balanced_dataset = pd.concat([dataset[dataset['Stance'] == minority_class]] +
                            [dataset[dataset['Stance'] == label].sample(minority_count, replace=True) for label in counts.index if label != minority_class])

# Mélanger le dataset équilibré
df = shuffle(balanced_dataset)

# Maintenant, balanced_dataset contient un nombre égal d'échantillons pour chaque classe dans la colonne 'label'


In [34]:
from sklearn.model_selection import train_test_split

# Séparation en features (X) et labels (y)
X = df[['articleBody', 'Headline']]
y = df['Stance']

# Utilisation de train_test_split pour diviser le dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Création des dataframes train_df et test_df
train_df = pd.DataFrame({'articleBody': X_train['articleBody'], 'Headline': X_train['Headline'], 'Stance': y_train})
test_df = pd.DataFrame({'articleBody': X_test['articleBody'], 'Headline': X_test['Headline'], 'Stance': y_test})


In [35]:
df.shape

(3360, 4)

In [36]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, concatenate
from imblearn.over_sampling import RandomOverSampler

# Supposons que vous ayez déjà défini train_df et test_df
# ...

# Créer un vecteurisateur Bag of Words
vectorizer = CountVectorizer(max_features=500)

# Transformer les textes en vecteurs
X_train_text = vectorizer.fit_transform(train_df['articleBody'])
X_test_text = vectorizer.transform(test_df['articleBody'])
X_train_headline = vectorizer.fit_transform(train_df['Headline'])
X_test_headline = vectorizer.transform(test_df['Headline'])


# Convertir les matrices CSR en matrices NumPy
X_train_text_np = X_train_text.toarray()
X_test_text_np = X_test_text.toarray()
X_train_headline_np = X_train_headline.toarray()
X_test_headline_np = X_test_headline.toarray()

# Convertir les labels en format numérique
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Créer le modèle
embedding_dim = 50
max_length = 100

input_text = Input(shape=(X_train_text_np.shape[1],))
embedding_text = Embedding(input_dim=500, output_dim=embedding_dim, input_length=X_train_text_np.shape[1])(input_text)
lstm_text = LSTM(100)(embedding_text)

input_headline = Input(shape=(X_train_headline_np.shape[1],))
embedding_headline = Embedding(input_dim=500, output_dim=embedding_dim, input_length=X_train_headline_np.shape[1])(input_headline)
lstm_headline = LSTM(100)(embedding_headline)

concatenated = concatenate([lstm_text, lstm_headline])
output = Dense(units=4, activation='softmax')(concatenated)

model = Model(inputs=[input_text, input_headline], outputs=output)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Entraîner le modèle avec les données converties en matrices NumPy
model.fit([X_train_text_np, X_train_headline_np], y_train_encoded, epochs=5, batch_size=64, validation_split=0.2)

# Évaluer le modèle
loss, accuracy = model.evaluate([X_test_text_np, X_test_headline_np], y_test_encoded)
print(f'Loss: {loss}, Accuracy: {accuracy}')


Epoch 1/5
34/34 [==============================] - 43s 1s/step - loss: 1.3875 - accuracy: 0.2409 - val_loss: 1.3878 - val_accuracy: 0.2268
Epoch 2/5
34/34 [==============================] - 35s 1s/step - loss: 1.3860 - accuracy: 0.2605 - val_loss: 1.3856 - val_accuracy: 0.2472
Epoch 3/5
34/34 [==============================] - 37s 1s/step - loss: 1.3836 - accuracy: 0.2656 - val_loss: 1.3811 - val_accuracy: 0.3011
Epoch 4/5
34/34 [==============================] - 32s 942ms/step - loss: 1.3762 - accuracy: 0.2940 - val_loss: 1.3793 - val_accuracy: 0.2732
Epoch 5/5
21/21 [==============================] - 3s 157ms/step - loss: 1.3723 - accuracy: 0.3065
Loss: 1.3723186254501343, Accuracy: 0.306547611951828


In [37]:
# Sélectionner les dix premières valeurs de votre dataframe de test
X_predict_text = vectorizer.transform(test_df['articleBody'].head(100)).toarray()
X_predict_headline = vectorizer.transform(test_df['Headline'].head(100)).toarray()

# Faire les prédictions
predictions = model.predict([X_predict_text, X_predict_headline])

# Convertir les prédictions en classes (labels)
predicted_classes = np.argmax(predictions, axis=1)

# Afficher les résultats
for i, pred_class in enumerate(predicted_classes):
    print(f"Exemple {i + 1}: Prédiction - {pred_class}, Classe réelle - {y_test_encoded[i]}")


4/4 [==============================] - 2s 103ms/step
Exemple 1: Prédiction - 2, Classe réelle - 3
Exemple 2: Prédiction - 1, Classe réelle - 3
Exemple 3: Prédiction - 2, Classe réelle - 3
Exemple 4: Prédiction - 3, Classe réelle - 0
Exemple 5: Prédiction - 2, Classe réelle - 2
Exemple 6: Prédiction - 2, Classe réelle - 2
Exemple 7: Prédiction - 2, Classe réelle - 3
Exemple 8: Prédiction - 2, Classe réelle - 1
Exemple 9: Prédiction - 1, Classe réelle - 0
Exemple 10: Prédiction - 2, Classe réelle - 1
Exemple 11: Prédiction - 2, Classe réelle - 3
Exemple 12: Prédiction - 1, Classe réelle - 1
Exemple 13: Prédiction - 2, Classe réelle - 2
Exemple 14: Prédiction - 2, Classe réelle - 3
Exemple 15: Prédiction - 2, Classe réelle - 2
Exemple 16: Prédiction - 2, Classe réelle - 3
Exemple 17: Prédiction - 2, Classe réelle - 2
Exemple 18: Prédiction - 2, Classe réelle - 2
Exemple 19: Prédiction - 2, Classe réelle - 1
Exemple 20: Prédiction - 2, Classe réelle - 2
Exemple 21: Prédiction - 2, Classe r

<h2><strong> Conclusion </strong></h2>

Les modèles de machine learning et de deep learning apprennent très peu. Cela pourrait être lié à l'une des raisons suivantes :
- prétraitement des données
- sac de mots (élimination de l'ordre des mots)
- modèles non optimisés